In [1]:
import jqdatasdk as jq
import pandas as pd

In [2]:
jq.auth('13918852005','jxjslsc,.?2')

auth success （JQData现有流量增加活动，详情请咨询JQData管理员，微信号：JQData01）


In [12]:
# 股指换手率、交易量数据 #
df = jq.finance.run_query(jq.query(jq.finance.STK_EXCHANGE_TRADE_INFO).filter(
    jq.finance.STK_EXCHANGE_TRADE_INFO.exchange_code==322001,
    jq.finance.STK_EXCHANGE_TRADE_INFO.date>='2009-01-01'
))

In [107]:
df.index = df['date']

In [109]:
df.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZZS_motion090101-190227.csv',index= True)

In [110]:
df1 = jq.finance.run_query(jq.query(jq.finance.STK_EXCHANGE_TRADE_INFO).filter(
    jq.finance.STK_EXCHANGE_TRADE_INFO.exchange_code==322005,
    jq.finance.STK_EXCHANGE_TRADE_INFO.date>='2009-01-01'
))
df1.index = df1['date']
df1.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/SZCZ_motion090101-190227.csv',index= True)

In [111]:
df2 = jq.finance.run_query(jq.query(jq.finance.STK_EXCHANGE_TRADE_INFO).filter(
    jq.finance.STK_EXCHANGE_TRADE_INFO.exchange_code==322007,
    jq.finance.STK_EXCHANGE_TRADE_INFO.date>='2009-01-01'
))
df2.index = df2['date']
df2.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/CYBZ_motion090101-190227.csv',index= True)

In [99]:
# 交易日前一日新闻联播内容股市相关提到次数 全跑一遍 #
# 关键词词库
df_temp0 = jq.finance.run_query(jq.query(jq.finance.CCTV_NEWS).filter(
        jq.finance.CCTV_NEWS.day>='2009-01-01',
        jq.finance.CCTV_NEWS.title.like('%%金融%%')
    ))

for key_word in ['股市','经济','银行','财政','银监会','证监会','投资','保监会']:
    df_temp1 = jq.finance.run_query(jq.query(jq.finance.CCTV_NEWS).filter(
        jq.finance.CCTV_NEWS.day>='2009-01-01',
        jq.finance.CCTV_NEWS.title.like('%%'+key_word+'%%')
    ))
    df_temp0 = pd.concat([df_temp0,df_temp1],axis=0)

In [100]:
df_new_cnt = df_temp0.groupby(df_temp0['day']).count()[['content']]

In [101]:
# 今天的新闻联播对明天有影响，因此日期+1
from datetime import datetime, date, timedelta
day_list_pre1 = []
for i in list(df_new_cnt.index):
    day_temp = i + timedelta(1)
    day_list_pre1.append(str(day_temp))

In [102]:
df_new_cnt.index = day_list_pre1

In [103]:
df_new_cnt_all = pd.DataFrame(columns=['temp'],index=pd.date_range('1/1/2009','2/1/2019'))
df_new_cnt_all = pd.merge(df_new_cnt_all, df_new_cnt, how = 'outer' ,left_index=True, right_index=True)
df_new_cnt_all = df_new_cnt_all[['content']]
df_new_cnt_all = df_new_cnt_all.fillna(0)

In [104]:
# datetime.strptime(df_new_cnt_all.index[i], '%Y-%m-%d').weekday() == 5 or datetime.strptime(df_new_cnt_all.index[i], '%Y-%m-%d').weekday() == 6

In [105]:
for i in range(len(df_new_cnt_all)):
    if df_new_cnt_all.index[i].weekday() == 5 or df_new_cnt_all.index[i].weekday() == 6:
        df_new_cnt_all.iloc[i+1,0] += df_new_cnt_all.iloc[i,0]
df_new_cnt_all.to_csv('C:/Users/jxjsj/Desktop/JupyterHome/Data/news_cnt_090101-190201.csv',index= True)  
# 最后用inner merge将df_new_cnt_all合并到总表中，会自动删除=5and=6(周六、周日新闻，留下周一（实际是周五+周六+周日）)